In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install roboflow

In [2]:
import pandas as pd
import re
import cv2
from roboflow import Roboflow
import os
import glob

In [3]:
dataset = '/content/drive/MyDrive/Inteligência artificial/CBIS-DDSM'

In [4]:
dicom_data = pd.read_csv(dataset + '/csv/dicom_info.csv')

### Calcificações

In [5]:
# patient_id, abnormality type, calc type, pathology, image file path

calc_case_description_train_set = pd.read_csv(dataset + '/csv/calc_case_description_train_set.csv',
                                              header=0,
                                              usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"])

calc_case_description_test_set = pd.read_csv(dataset + '/csv/calc_case_description_test_set.csv',
                                             header=0,
                                             usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"])


### Massas

In [12]:
# patient_id, abnormality type, mass shape, pathology, image file path

mass_case_description_train_set = pd.read_csv(dataset + '/csv/mass_case_description_test_set.csv',
                                              header=0,
                                              usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"])

mass_case_description_test_set = pd.read_csv(dataset + '/csv/mass_case_description_test_set.csv',
                                                                                           header=0,
                                              usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"])


In [13]:
def alterar_nome_da_pasta(nome):
  teste = "Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.289610447411344525237308079592285912683/1.3.6.1.4.1.9590.100.1.2.22131189612893294827907969600765582967/000000.dcm"

  padrao = r'.*?/.*?/(?P<string>.*?)/'
  resultado = re.search(padrao, nome)

  string_desejada = resultado.group('string')
  print(string_desejada)

  return string_desejada

In [ ]:
mass_case_description_train_set['image file path']

### Alterando o caminho das pastas

In [15]:
calc_case_description_train_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

calc_case_description_test_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

mass_case_description_train_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

mass_case_description_test_set['image file path'] = mass_case_description_test_set['image file path'].apply(alterar_nome_da_pasta)


1.3.6.1.4.1.9590.100.1.2.245063149211255120613007755642780114172
1.3.6.1.4.1.9590.100.1.2.85952214611170506017891429690540035518
1.3.6.1.4.1.9590.100.1.2.22131189612893294827907969600765582967
1.3.6.1.4.1.9590.100.1.2.239949064412092068706566726490415129934
1.3.6.1.4.1.9590.100.1.2.215081818713600536113960661873725083371
1.3.6.1.4.1.9590.100.1.2.318799084911119262430780458250312419361
1.3.6.1.4.1.9590.100.1.2.345140832810160378520078721331878282316
1.3.6.1.4.1.9590.100.1.2.59887648512379053809277845872128232362
1.3.6.1.4.1.9590.100.1.2.302678903112208607702983192631368637142
1.3.6.1.4.1.9590.100.1.2.398695186611278204714097118000162925433
1.3.6.1.4.1.9590.100.1.2.334704328611391643805508991083300190349
1.3.6.1.4.1.9590.100.1.2.188340832412548997809157866361536031816
1.3.6.1.4.1.9590.100.1.2.17873451612287715806233693563532897726
1.3.6.1.4.1.9590.100.1.2.18321540012684193708096260002352282494
1.3.6.1.4.1.9590.100.1.2.18321540012684193708096260002352282494
1.3.6.1.4.1.9590.100.1.2.170208

In [ ]:
mass_case_description_train_set['image file path']

### Cria o ROI

In [ ]:
def obter_nomes_imagens(diretorio, extensoes=['*.jpg']):
    caminho_completo = os.path.join(diretorio, '*')
    nomes_imagens = []

    for extensao in extensoes:
        caminho_imagens = os.path.join(diretorio, extensao)
        nomes_imagens.extend(glob.glob(caminho_imagens))

    nomes_imagens = [os.path.basename(imagem) for imagem in nomes_imagens]
    return nomes_imagens

def cortar_imagem(nome_pasta):
    rf = Roboflow(api_key="CEfThkqYjDhVzpzg2wJO")
    project = rf.workspace().project("breast-cancer-roi-n4ssp")
    model = project.version(1).model

    # print(model.predict("/content/1-126.jpg", confidence=50, overlap=50).json())

    nome_pasta = dataset + '/jpeg/' + nome_pasta
    nomes_imagens = obter_nomes_imagens(nome_pasta)

    prediction = model.predict(nome_pasta + '/' + nomes_imagens[0], confidence=50, overlap=50).json()
    prediction_list = prediction["predictions"]
    prediction_dict = prediction_list[0]

    roi_x = int(prediction_dict['x'] - prediction_dict['width'] / 2)
    roi_y = int(prediction_dict['y'] - prediction_dict['height'] / 2)
    roi_width = int(prediction_dict['width'])
    roi_height = int(prediction_dict['height'])

    image = cv2.imread(nome_pasta + '/' + nomes_imagens[0])

    roi = image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    # print(roi)

    cv2.imwrite(nome_pasta + "/roi_" + nomes_imagens[0], roi)

# falta chamar essa função para todas as pastas de treinamento e teste
nome_pasta = '1.3.6.1.4.1.9590.100.1.2.126082211045731020508108042042916052'
cortar_imagem(nome_pasta)